In [4]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [5]:
def Unitvector(x_gauss, y_gauss):
    from sklearn import preprocessing
    
    # get the slope of the tangent
    trajectory = np.asarray([x_gauss, y_gauss])
    m = np.gradient(trajectory, axis = 1)
    m_atx = m[1]/m[0]
    
    # get the tangent vector at x = x0 + 1
    tangent_x = x_gauss+1
    tangent_y = m_atx + y_gauss
    
    # get the unit tangent vector
    u_x = []
    u_y = []

    for x,y,x0,y0 in zip(tangent_x, tangent_y, x_gauss, y_gauss):
        if np.any(np.isnan([x, y])) or np.any(np.isinf([x, y])):
            unit_x = np.nan
            unit_y = np.nan
        else:
            vector = np.asarray([x-x0, y-y0]).reshape(1,-1)
            [unit_x, unit_y] = preprocessing.normalize(vector, norm = 'l2')[0]
        u_x.append(unit_x)
        u_y.append(unit_y)
    u_x = np.asarray(u_x)
    u_y = np.asarray(u_y)
    return(u_x, u_y)

def getAngle(loc, tangent):
    if np.any(np.isnan([loc, tangent])):
        angle = np.nan
    else:
        cross = np.cross(tangent, loc)
        dot = np.dot(tangent, loc)
        angle = np.arctan2(cross, dot)*180/np.pi
    return(angle)

def wrapAngle(angle):
    angle = np.absolute(angle)
    for i,a in enumerate(angle):
        if a > 90:
            a = 180 - a
            angle[i] = a
    return(angle)

In [6]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [7]:
visitnum = ['FirstVisit/', 'Later7thVisit/','Later20thVisit/', 'LastVisit/']

## get angle and magnitude

In [8]:
for visit in visitnum:
    direc = os.path.join(r"../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/",
                         visit)

    trackslist = glob.glob(direc + 'c*.csv')

    newpath = os.path.join(r"../dataFolders/PaperPipelineOutput/v3/RadiusAndAngle/", visit)
    if not os.path.exists(newpath):
        os.mkdir(newpath)
    
    for data in trackslist:
        moth = os.path.basename(data)[:-4]

        segments = moth.split("_")
        moth = segments[0] + '_' + segments[1]

        file = pd.read_csv(data)

#         file = file.dropna()

        x = file.x.values
        y = file.y.values

        # find the circ parameters that corresponds to that moth
        for row, n in enumerate(circ_parameters.name):
            if moth == n:
                r = row
            elif moth + '_cropped' == n:
                r = row
                
        circ_x = circ_parameters.loc[r,'circ_x']
        circ_y = circ_parameters.loc[r, 'circ_y']
        circ_radii = circ_parameters.loc[r, 'circ_radii']
#         print(moth)
#         print(circ_parameters.loc[r, 'name'])

        x = x - circ_x
        y = y - circ_y
        r = np.linalg.norm([x, y], axis = 0)/circ_radii

        # save all usable variables as series

        df1 = pd.Series(data = x, name = 'x_centered')
        df2 = pd.Series(data = y, name = 'y_centered')
        df3 = pd.Series(data = r, name = 'radial_distance_normalized')


        #calculate the unit tangent vectors - savitzky-golay vector
        u_x, u_y = Unitvector(x, y)

        angle = []
        for x0, y0, x, y in zip(x, y, u_x, u_y):
            loc = [x0, y0]
            tangent = [x, y]
            a = getAngle(loc, tangent)
            angle.append(a)

        angle = wrapAngle(angle)

        df4 = pd.Series(data = angle, name = 'angle')

        new_file = pd.concat([df1, df2, df3, df4], axis = 1)
        new_file.to_csv(newpath + moth + '_' + visit[:-1] + '_RadiusAndAngle.csv', index_label = False)

C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\L

C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
